# pyLDAvis

[`pyLDAvis`](https://github.com/bmabey/pyLDAvis) is a python libarary for interactive topic model visualization.
It is a port of the fabulous [R package](https://github.com/cpsievert/LDAvis) by Carson Sievert and Kenny Shirley.  They did the hard work of crafting an effective visualization. `pyLDAvis` makes it easy to use the visualiziation from Python and, in particualr, IPython notebooks. To learn more about the method behind the visualization I suggest reading the [original paper](http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf) explaining it.

This notebook provides a quick overview of how to use `pyLDAvis`. Refer to the [documenation](https://pyldavis.readthedocs.org/en/latest/) for details.


## BYOM - Bring your own model

`pyLDAvis` is agnostic to how your model was trained. To visualize it you need to provide the topic-term distribtuions, document-topic distributions, and basic information about the corpus which the model was trained on. The main function is the [`prepare`](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.prepare) function that will transform your data into the format needed for the visualization.

Below we load a model trained in R and then visualize it. The model was trained on a corpus of 2000 movie reviews parsed by [Pang and Lee (ACL, 2004)](http://www.cs.cornell.edu/people/pabo/movie-review-data/), originally gathered from the IMDB archive of the rec.arts.movies.reviews newsgroup.

In [1]:
import json
import numpy as np

def load_R_model(filename):
    with open(filename, 'r') as j:
        data_input = json.load(j)
    data = {'topic_term_dists': data_input['phi'], 
            'doc_topic_dists': data_input['theta'],
            'doc_lengths': data_input['doc.length'],
            'vocab': data_input['vocab'],
            'term_frequency': data_input['term.frequency']}
    return data

movies_model_data = load_R_model('data/movie_reviews_input.json')

print('Topic-Term shape: %s' % str(np.array(movies_model_data['topic_term_dists']).shape))
print('Doc-Topic shape: %s' % str(np.array(movies_model_data['doc_topic_dists']).shape))

Topic-Term shape: (20, 14567)
Doc-Topic shape: (2000, 20)


Now that we have the data loaded we use the `prepare` function:

In [2]:
import pyLDAvis
movies_vis_data = pyLDAvis.prepare(**movies_model_data)

Once you have the visualization data prepared you can do a number of things with it. You can [save the vis](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.save_html) to an stand-alone HTML file, [serve it](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.show), or [dispaly it](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.display) in the notebook. Let's go ahead and display it:

In [3]:
pyLDAvis.display(movies_vis_data)

Pretty, huh?! Again, you should be thanking the original [LDAvis people](https://github.com/cpsievert/LDAvis) for that. You may thank me for the IPython integartion though. :) Aside from being aesthetically pleasing this visualization more importantly represents a lot of information about the topic model that is hard to take in all at once with ad-hoc queries. To learn more about the visual elements and how they help you explore your model see [this documentation](http://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf) from the original R project and this presentation ([slides](https://speakerdeck.com/bmabey/visualizing-topic-models), [video](https://www.youtube.com/watch?v=tGxW2BzC_DU)).


To see other models visualzied check out [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Movie%20Reviews,%20AP%20News,%20and%20Jeopardy.ipynb).

*ProTip:* To avoid tediously typing in `display` all the time use:

In [4]:
pyLDAvis.enable_notebook()

## Making the common case easy - Gensim and others!

Built on top of the generic `prepare` function are helper functions for [gensim](https://radimrehurek.com/gensim/) and [GraphLab Create](https://dato.com/products/create/). To demonstrate below I am loading up a trained gensim model and coresponding dictionary and corpus (see [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Gensim%20Newsgroup.ipynb) for how these were created):

In [6]:
import gensim

dictionary = gensim.corpora.Dictionary.load('newsgroups.dict')
corpus = gensim.corpora.MmCorpus('newsgroups.mm')
lda = gensim.models.ldamodel.LdaModel.load('newsgroups_50.model')

/Users/bmabey/anaconda/envs/data/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In the dark ages in order to inspect our topics all we had was `show_topics` and friends:

In [7]:
lda.show_topics()

[(37,
  u'0.018*food + 0.018*msg + 0.014*disease + 0.011*henry + 0.010*patients + 0.010*cancer + 0.010*health + 0.010*doctor + 0.007*eat + 0.007*effects'),
 (18,
  u'0.038*men + 0.034*constitution + 0.021*male + 0.014*purdue + 0.013*partners + 0.012*study + 0.012*women + 0.011*cross + 0.009*female + 0.009*weaver'),
 (0,
  u'0.028*host + 0.027*nntp + 0.027*posting + 0.025*newsreader + 0.021*tin + 0.020*technology + 0.019*institute + 0.017*michael + 0.017*edu + 0.016*keith'),
 (17,
  u'0.013*states + 0.011*new + 0.010*american + 0.009*state + 0.008*national + 0.008*united + 0.008*congress + 0.008*drugs + 0.007*press + 0.007*washington'),
 (33,
  u"0.025*year + 0.024*players + 0.021*league + 0.018*team + 0.015*division + 0.015*boston + 0.013*season + 0.013*hit + 0.011*fan + 0.011*he's"),
 (7,
  u'0.087*car + 0.033*cars + 0.030*andrew + 0.012*mellon + 0.012*carnegie + 0.011*mph + 0.011*ford + 0.009*cmu + 0.009*pittsburgh + 0.008*fpu'),
 (12,
  u'0.026*colorado + 0.020*fbi + 0.015*jim + 0.0

Thankfully, in addition to these *still helpful functions*, we can get a feel for all of the topics with this one-liner:

In [8]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
41     13.532437        1       1 -0.379508 -0.029962
23     10.074164        1       2 -0.315505 -0.098711
36      8.194614        1       3 -0.254861 -0.120593
47      7.374750        1       4 -0.211872  0.119316
22      4.927411        1       5 -0.239426  0.156840
25      4.604632        1       6 -0.022528 -0.039393
39      4.534610        1       7 -0.215478 -0.146610
1       3.465174        1       8 -0.196831 -0.042025
17      2.809805        1       9 -0.104767 -0.098462
10      2.566919        1      10 -0.163740  0.058192
9       2.098628        1      11 -0.115900  0.217352
15      2.031405        1      12 -0.143551 -0.072431
4       1.884474        1      13 -0.098123  0.035985
29      1.876140        1      14 -0.028189  0.100614
30      1.840968        1      15 -0.054077 -0.152326
11      1.768139        1      16 -0.072649  0.127677
44      1.754696        1      17  0.053037 -0.023930
31      1.735248        1      18 -0.033997 -0.100836
16      1.648157        1      19 -0.058918 -0.140544
35      1.448246        1      20 -0.060875  0.096865
45      1.393123        1      21 -0.008901  0.067532
14      1.317507        1      22  0.010791  0.004059
49      1.271832        1      23 -0.037673  0.158739
34      1.205319        1      24 -0.004220  0.141175
33      1.203547        1      25  0.031516 -0.058999
32      1.038478        1      26  0.067543 -0.067880
0       0.992931        1      27 -0.006161  0.085689
37      0.964599        1      28  0.088435 -0.023948
19      0.909300        1      29  0.084763 -0.016447
40      0.851704        1      30  0.063318 -0.010460
46      0.823040        1      31  0.109703 -0.012693
12      0.736923        1      32  0.073438  0.000404
7       0.687919        1      33  0.114525 -0.013366
42      0.512332        1      34  0.115533  0.002870
28      0.510050        1      35  0.111197 -0.005060
3       0.453143        1      36  0.122420  0.006250
38      0.451958        1      37  0.119801 -0.012281
27      0.445288        1      38  0.108083  0.008028
26      0.428628        1      39  0.122761 -0.009456
20      0.388130        1      40  0.131827 -0.009134
6       0.385954        1      41  0.130388 -0.015161
13      0.371847        1      42  0.126191 -0.006191
24      0.364411        1      43  0.123861 -0.000607
18      0.360866        1      44  0.124773 -0.021908
48      0.339396        1      45  0.121509 -0.001543
2       0.304238        1      46  0.135028 -0.007851
21      0.296836        1      47  0.132325 -0.003900
5       0.291829        1      48  0.131519 -0.006824
8       0.291696        1      49  0.135574 -0.010789
43      0.236558        1      50  0.137888 -0.007264, topic_info=      Category          Freq          Term  Total  loglift  logprob
8169   Default  62049.000000           'ax  62049  30.0000  30.0000
11689  Default   4771.000000           max   4771  29.0000  29.0000
10296  Default   5498.000000       posting   5498  28.0000  28.0000
12191  Default   4809.000000          nntp   4809  27.0000  27.0000
15194  Default   4991.000000          host   4991  26.0000  26.0000
9147   Default   5585.000000    university   5585  25.0000  25.0000
12657  Default   2006.000000          file   2006  24.0000  24.0000
2059   Default   7436.000000       article   7436  23.0000  23.0000
18697  Default   1616.000000         drive   1616  22.0000  22.0000
19661  Default   3370.000000           edu   3370  21.0000  21.0000
6692   Default   1844.000000         space   1844  20.0000  20.0000
6125   Default   5850.000000        people   5850  19.0000  19.0000
9569   Default   1666.000000           key   1666  18.0000  18.0000
6484   Default   1292.000000           car   1292  17.0000  17.0000
15840  Default   2621.000000           god   2621  16.0000  16.0000
18944  Default   1951.000000       windows   1951  15.0000  1

## GraphLab

As I mentioned above you can also easily visualize GraphLab TopicModels as well. Check out [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/GraphLab.ipynb#topic=7&lambda=0.41&term=) if you are interested in that.


## Go forth and visualize!

What are you waiting for? Go ahead and `pip install pyldavis`.